Copyright (c) Facebook, Inc. and its affiliates.
All rights reserved.

This source code is licensed under the license found in the
LICENSE file in the root directory of this source tree.

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.sparse import csc_matrix

import time
import imp

import cvxpy
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels

# SPPE Code

In [ ]:
def get_objective(objective, s, h, n, m):
    if objective == "max_revenue":
        return xp.Sum(s[i][j] for j in range(m) for i in range(n))
    elif objective == "min_revenue":
        return xp.Sum(-s[i][j] for j in range(m) for i in range(n))
    elif objective == "min_paced_welfare":
        return xp.Sum(-h[j] for j in range(m))
    else: # max paced welfare
        return xp.Sum(h[j] for j in range(m))
    
def sppe(V, B, objective = "max_revenue"):
    n,m = V.shape
    
    alpha = [xp.var(lb=0, ub=1) for i in range(n)] # pacing multiplier
    y = [xp.var(vartype=xp.binary, lb=0, ub=1) for i in range(n)] # if buyer i spends its full budget
    
    p = [xp.var(lb=0) for j in range(m)] # prices
    h = [xp.var(lb=0) for j in range(m)] # highest bid for good j
    

    s = [[xp.var(lb=0) for j in range(m)] for i in range(n)] # buyers i's spend on good j
    
    d = [[xp.var(vartype=xp.binary, lb=0, ub=1) for j in range(m)] for i in range(n)] # 1 if buyer i may win part of good j
    w = [[xp.var(vartype=xp.binary, lb=0, ub=1) for j in range(m)] for i in range(n)] # 1 if buyer i is the winner of good j
    r = [[xp.var(vartype=xp.binary, lb=0, ub=1) for j in range(m)] for i in range(n)] # 1 if buyer i is the second price for good j
    
    model = xp.problem ("sppe")
    model.addVariable(alpha, y, p, h, s, d, w, r)
    
    # constraints in same order as here: https://arxiv.org/abs/1706.07151v3
    # budget constraint
    model.addConstraint(xp.Sum(s[i][j] for j in range(m)) <= B[i] for i in range(n)) # (1)
    # budget complementarity constraints
    model.addConstraint(xp.Sum(s[i][j] for j in range(m)) >= y[i] * B[i] for i in range(n)) # (2)
    model.addConstraint(alpha[i] >= 1-y[i] for i in range(n)) # (3)
    # spends on good should equal price of good
    model.addConstraint(xp.Sum(s[i][j] for i in range(n)) == p[j] for j in range(m)) # (4)
    # 
    model.addConstraint(s[i][j] <= B[i]*d[i][j] for j in range(m) for i in range(n)) # (5)
    model.addConstraint(h[j] >= alpha[i]*V[i,j] for j in range(m) for i in range(n)) # (6)
    model.addConstraint(h[j] <= alpha[i]*V[i,j] + np.max(V[:,j]) - np.max(V[:,j])*d[i][j] for j in range(m) for i in range(n)) # (7)
    model.addConstraint(w[i][j] <= d[i][j] for j in range(m) for i in range(n)) # (8)
    model.addConstraint(p[j] >= alpha[i]*V[i,j] - V[i,j]*w[i][j] for j in range(m) for i in range(n)) # (9)
    model.addConstraint(p[j] <= alpha[i]*V[i,j] + np.max(V[:,j]) - np.max(V[:,j])*r[i][j] for j in range(m) for i in range(n)) # (10)

    model.addConstraint(xp.Sum(w[i][j] for i in range(n)) == 1 for j in range(m)) # (11)
    model.addConstraint(xp.Sum(r[i][j] for i in range(n)) == 1 for j in range(m)) # (12)

    model.addConstraint(r[i][j] + w[i][j] <= 1 for j in range(m) for i in range(n)) # (13)
    
    obj = get_objective(objective, s, h, n, m)
    model.setObjective(obj, sense=xp.maximize)
    t0 = time.time()
    model.solve()
    runtime = time.time() - t0

    prices = model.getSolution(p)
    multipliers = model.getSolution(alpha)
    allocation = np.zeros((n,m), dtype=np.float)
    spends = np.zeros((n,m), dtype=np.float)
    for i in range(n):
        for j in range(m):
            spends[i,j] = model.getSolution(s[i][j])
            if prices[j] > 0:
                allocation[i,j] = model.getSolution(s[i][j]) / prices[j]
            else:
                allocation[i,j] = model.getSolution(w[i][j])

    return multipliers, prices, allocation, runtime

# FPPE Code

In [ ]:
def eisenberg_gale(valuations, B=None):
    if np.min(valuations[:, 2]) < 1e-7:
        warnings.warn('The provided valuations are badly scaled.')

    num_buyers = int(np.max(valuations[:, 0])) + 1
    num_items = int(np.max(valuations[:, 1])) + 1
    num_pairs = valuations.shape[0]

    if B is None:
        B = np.ones(num_buyers)
    elif B.size != num_buyers:
        raise RuntimeError('The provided budget vector is of the wrong size.')
    elif np.min(B) < 0:
        raise RuntimeError('The provided budget vector has a negative entry.')

    supply = np.ones(num_items)

    P_rows = valuations[:, 1].astype(int)
    P_cols = np.arange(num_pairs)
    P_vals = np.ones(shape=(num_pairs,))
    P = csc_matrix((P_vals, (P_rows, P_cols)))

    U_rows = valuations[:, 0].astype(int)
    U_cols = np.arange(num_pairs)
    U_vals = valuations[:, 2]
    U = csc_matrix((U_vals, (U_rows, U_cols)))

    x = cvxpy.Variable(shape=(num_pairs,), name='x')
    constraints = [P @ x <= supply]
    constraints.append(x >= 0)
    l = cvxpy.Variable(shape=(num_buyers,), nonneg=True, name='l')
    objective = cvxpy.Maximize(B @ cvxpy.log(U @ x + l) - cvxpy.sum(l))

    prob = cvxpy.Problem(objective, constraints)
    prob.__dict__['metadata'] = {'capacities': supply, 'P': P, 'U': U}
    return prob


def items_per_buyer(valuations, skip_sort=False):
    if not skip_sort:
        valuations = valuations[np.argsort(valuations[:, 0]), :]
    d = np.diff(np.hstack([[-1], valuations[:, 0]]))
    startlocs = np.where(d == 1)[0]
    # ^ starting index for each buyer
    startlocs = np.hstack([startlocs, [valuations.shape[0]]])
    # ^ add imaginary "final" buyer
    itemct = np.diff(startlocs)
    return itemct


def buyers_per_item(valuations, skip_sort=False):
    if not skip_sort:
        valuations = valuations[np.argsort(valuations[:, 1]), :]
    d = np.diff(np.hstack([[-1], valuations[:, 1]]))
    startlocs = np.where(d == 1)[0]
    # ^ starting index for each item, in this sorted order
    startlocs = np.hstack([startlocs, [valuations.shape[0]]])
    # ^ add imaginary "final" item
    buyerct = np.diff(startlocs)
    return buyerct


def results_by_item(valuations, allocations, prices):
    order = np.argsort(valuations[:, 1])
    valuations = valuations[order, :]
    allocations = allocations[order]
    widths = buyers_per_item(valuations, skip_sort=True)
    start = 0
    item2result = []
    for i, w in enumerate(widths):
        stop = start + w
        curr_buyers = valuations[start:stop, 0].astype(int)
        curr_values = valuations[start:stop, 2]
        curr_allocs = allocations[start:stop]
        d = {'buyers': curr_buyers,
             'valuations': curr_values,
             'allocations': curr_allocs,
             'price': prices[i] if prices is not None else None}
        item2result.append(d)
        start = stop
    return item2result


def results_by_buyer(valuations, allocations, prices):
    order = np.argsort(valuations[:, 0])
    valuations = valuations[order, :]
    allocations = allocations[order]
    widths = items_per_buyer(valuations, skip_sort=True)
    start = 0
    buyer2result = []
    for w in widths:
        stop = start + w  # inclusive of "start + w".
        curr_items = valuations[start:stop, 1].astype(int)
        curr_value = valuations[start:stop, 2]
        curr_alloc = allocations[start:stop]
        curr_prices = prices[curr_items] if prices is not None else None
        d = {'items': curr_items,
             'valuations': curr_value,
             'allocations': curr_alloc,
             'prices': curr_prices}
        buyer2result.append(d)
        start = stop
    return buyer2result


def convert_dense_valuation(v):
    num_buyers = v.shape[0]
    num_items = v.shape[1]
    buyer_index_vec = np.repeat(np.arange(num_buyers), num_items)
    item_index_vec = np.tile(np.arange(num_items), num_buyers)
    utility_vec = np.ravel(v, order='C')  # C, versus Fortran order.
    # convert the above vectors into columns
    buyer_index_vec = buyer_index_vec.reshape((-1, 1))
    item_index_vec = item_index_vec.reshape((-1, 1))
    utility_vec = utility_vec.reshape((-1, 1))
    # horizontally stack those to obtain a matrix in the required format.
    valuations = np.hstack([buyer_index_vec, item_index_vec, utility_vec])
    return valuations


def fppe(V, b):
    vals = convert_dense_valuation(V)
    prob = eisenberg_gale(vals, b)
    obj = prob.solve(verbose=False)
    # obj = prob.solve(solver='MOSEK', verbose=True)
    runtime = prob.solver_stats.solve_time

    model_variables = prob.variables()
    x = model_variables[0].value
    capacity_constraint = prob.constraints[0]
    p = capacity_constraint.dual_value

    item_res = results_by_item(vals, x, p)
    buyer_res = results_by_buyer(vals, x, p)
    allocation = np.zeros(V.shape)
    for i in range(V.shape[0]):
        for res_idx in range(buyer_res[i]["items"].size):
            j = buyer_res[i]["items"][res_idx]
            allocation[i, j] = buyer_res[i]["allocations"][res_idx]
            
    alpha = b / ((V*allocation).sum(axis=1) + b - allocation@p)
    return alpha, p, allocation, runtime


# Generate Instances

In [ ]:
#settings
advertiser_set = [10, 20, 30, 40, 100]
user_set = [10, 20, 30, 100]
instances_per_setting = 1

for num_advertisers in advertiser_set:
    for num_users in user_set:
        for i in range(instances_per_setting):
            V = np.random.uniform(size=(num_advertisers, num_users))
            B = np.random.uniform(high=num_users / num_advertisers, size=num_advertisers)
            
            path_str = f'./data/num_advertisers_{num_advertisers}_num_users_{num_users}_id_{i}'
            try:
                os.makedirs(path_str)
            except OSError:
                print("folder already exists, overwriting old instance")
                
            np.save(path_str + "/V", V)
            np.save(path_str + "/B", B)

# Load Instances

In [ ]:
instance_list = [f.path for f in os.scandir('./data/') if f.is_dir()]

In [ ]:
instances = {}
for path_str in instance_list:
    V = np.load(path_str + "/V.npy")
    B = np.load(path_str + "/B.npy")
    
    instances[path_str] = (V,B)

# Bid/Multiplier Deviations (Figures 1 and 4)

In [ ]:
df = pd.DataFrame(columns=["name",
                           "num_advertisers",
                           "ad_id",
                           "num_users",
                           "fppe_util",
                           "fppe_value",
                           "fppe_alpha",
                           "best_bids_util",
                           "best_bids_value",
                           "best_alpha",
                           "best_alpha_util",
                           "best_alpha_value"],
                   dtype=np.float32)
df["name"] = df["name"].astype('string')
df["num_advertisers"] = df["num_advertisers"].astype('uint8')
df["ad_id"] = df["ad_id"].astype('uint8')
df["num_users"] = df["num_users"].astype('uint8')

In [ ]:
for name in instances:
    (V,B) = instances[name]
    try:
        alpha, p, x, rt = fppe(V,B)
    except:
        continue
        
    num_advertisers, num_users = V.shape
    
    for bidder in range(num_advertisers):    
        # current utility
        curr_util = sum(x[bidder] * (V[bidder] - p))
        curr_value = sum(x[bidder] * V[bidder])
    
        # compute prices for items with bidder i
        paced_bids = alpha.reshape(num_advertisers,1) * V
        paced_bids[bidder,:] = np.zeros(num_users)
        prices = paced_bids.max(axis=0)
        
        # find best deviations
        items = [(V[bidder, i], prices[i], V[bidder, i]/prices[i]) for i in range(num_users)]
        items.sort(key=lambda tup: tup[2], reverse=True)
        
        # find best individual bids
        cost = 0.
        value = 0.

        for item in items:
            if item[2] > 1.:
                spend = min(item[1], B[bidder] - cost) # how much will we spend on this item
                value += item[0] * min(max(spend / item[1], 0.0), 1.0) # added value of quantity we bought
                cost += spend # update total spend
                if spend <= 0.00001:
                    break
                    
        best_bid_util = value - cost
        best_bid_value = value

        
        # find best shading multiplier
        max_util = 0.
        max_value = 0.
        best_alpha = 0.
        for i in range(len(items)):
            if items[i][2] >= 1.:
                multiplier = min(max(1. / items[i][2], 0.0), 1.0)
                value = 0.
                cost = 0.
                for j in range(i+1):
                    price = items[j][0]*multiplier
                    spend = min(price, B[bidder] - cost)
                    value += items[j][0] * min(max(spend / price, 0.0), 1.0)
                    cost += spend
                    if spend <= 0.000001:
                        break
                util = value - cost
                if util > max_util:
                    max_util = max(max_util, util)
                    max_value = value
                    best_alpha = multiplier
        best_multiplier_util = max_util
        best_multiplier_value = max_value

        
        df.loc[df.ad_id.count()] = {"name": name,
            "num_advertisers": num_advertisers,
                   "ad_id": bidder,
                   "num_users": num_users,
                   "fppe_util": curr_util,
                   "fppe_value": curr_value,
                   "fppe_alpha": alpha[bidder],
                   "best_bids_util": best_bid_util,
                   "best_bids_value": best_bid_value,
                   "best_alpha": best_alpha,
                   "best_alpha_util": best_multiplier_util,
                   "best_alpha_value": best_multiplier_value}
        

df.to_pickle('bid_budget_deviations_df.zip')

## Visualizations

In [ ]:
df2 = pd.read_pickle('bid_budget_deviations_df.zip')
df2["rel_regret_bids"] = 1.0 - df2["fppe_util"] / df2["best_bids_util"]
df2["rel_regret_bids"] = df2["rel_regret_bids"].clip(lower=0., upper=1.)
df2["rel_regret_alpha"] = 1.0 - df2["fppe_util"] / df2["best_alpha_util"]
df2["rel_regret_alpha"] = df2["rel_regret_alpha"].clip(lower=0., upper=1.)
df2["rel_regret_alpha_value"] = (df2["best_alpha_util"] - df2["fppe_util"]) / df2["best_alpha_value"]
df2["rel_regret_alpha_value"] = df2["rel_regret_alpha_value"].clip(lower=0., upper=1.)

In [ ]:
g1a =sns.catplot(data = df2,
    x = "num_advertisers",
    y = "rel_regret_bids",
    hue = "num_users",
    fliersize=0.25,
    kind = "box",
    aspect =1.3)
(g1a.set_axis_labels("Number of Advertisers", "Max Relative Regret\n(Alternative Bids)")
  ._legend.set_title("Number\nof Users"))
try:
    os.makedirs("./figs")
except OSError:
    a =1
    
plt.savefig("./figs/1a.pdf")

In [ ]:
g1b = sns.catplot(data = df2,
    x = "num_advertisers",
    y = "rel_regret_alpha",
    hue = "num_users",
    fliersize=0.25,
    kind = "box",
    aspect = 1.3)
(g1b.set_axis_labels("Number of Advertisers", "Max Relative Regret\n(Alternative Multiplier)")
  ._legend.set_title("Number\nof Users"))
plt.savefig("./figs/1b-1.pdf")

(g1b.set(ylim=(0, .1)))
plt.savefig("./figs/1b-2.pdf")

In [ ]:
df2a = df2[df2["num_advertisers"]>=8]
#df2a = df2a[df2a["num_users"] >= 10]

g4 = sns.lmplot(x="fppe_alpha", y="rel_regret_alpha", data=df2a, scatter_kws={'alpha':0.1, 'color':'k'}, lowess=True)
(g4.set_axis_labels("Pacing Multiplier", "Max Relative Regret\n(Alternative Multiplier)"))
plt.savefig("./figs/4.pdf")

# Input Deviations (Figure 2)

In [ ]:
df_input_dev = pd.DataFrame(columns=["name",
                           "num_advertisers",
                           "ad_id",
                           "num_users",
                           "fppe_util",
                           #"budget_multiplier",
                           #"bid_multiplier",
                           "dev_util"],
                   dtype=np.float32)
df_input_dev["name"] = df_input_dev["name"].astype('string')
df_input_dev["num_advertisers"] = df_input_dev["num_advertisers"].astype('uint8')
df_input_dev["ad_id"] = df_input_dev["ad_id"].astype('uint8')
df_input_dev["num_users"] = df_input_dev["num_users"].astype('uint8')

In [ ]:
budget_multipliers = [0.1 * i for i in range(1, 11)]
bid_multipliers = [0.1 * i for i in range(1, 12)]

In [ ]:
num_instances = len(instances)
iteration = 0

for name in instances:
    (V,B) = instances[name]
    try:
        alpha, p, x, rt = fppe(V,B)
    except:
        continue
        
    num_advertisers, num_users = V.shape
    
    for bidder in range(num_advertisers):    
        # current utility
        curr_util = sum(x[bidder] * (V[bidder] - p))
    
        alt_util = curr_util
        
        V_prime = np.copy(V)
        B_prime = np.copy(B)
        for bid_m in bid_multipliers:
            V_prime[bidder] = V[bidder]*bid_m
            for bud_m in budget_multipliers:
                B_prime[bidder] = B[bidder] * bud_m

                try:
                    alpha_prime, p_prime, x_prime, rt_prime = fppe(V_prime,B_prime)
                    alt_util = max(alt_util, sum(x_prime[bidder] * (V[bidder] - p_prime)))
                except:
                    continue
        
        
        df_input_dev.loc[df_input_dev.ad_id.count()] = {"name": name,
                "num_advertisers": num_advertisers,
                "ad_id": bidder,
                "num_users": num_users,
                "fppe_util": curr_util,
                "dev_util": alt_util}
    print(f"Done with {iteration} of {num_instances}!")
    iteration += 1
        

In [ ]:
df_input_dev.to_pickle('fppe_deviations_df.zip')

## Visualization

In [ ]:
df2_fppe = pd.read_pickle('fppe_deviations_df.zip')
df2_fppe["rel_regret"] = 1.0 - df2_fppe["fppe_util"] / df2_fppe["dev_util"]
df2_fppe["rel_regret"] = df2_fppe["rel_regret"].clip(lower=0., upper=1.)

In [ ]:
g2 = sns.catplot(data = df2_fppe,
    x = "num_advertisers",
    y = "rel_regret",
    hue = "num_users",
    fliersize=0.25,
    kind = "box",
    aspect = 1.5)
(g2.set_axis_labels("Number of Advertisers", "Relative Regret\n(Alternative Pacing Inputs)")
  ._legend.set_title("Number\nof Users"))
plt.savefig("./figs/2.pdf")

(g2.set(ylim=(0, .1)))
plt.savefig("./figs/2-2.pdf")

# SPPE/FPPE Comparisons (Figure 3)

In [ ]:
df = pd.DataFrame(columns=["name",
                           "num_advertisers",
                           "num_users",
                           "fppe_welfare",
                           "fppe_rev",
                           "sppe_welfare",
                           "sppe_rev"],
                   dtype=np.float32)
df["name"] = df["name"].astype('string')
df["num_advertisers"] = df["num_advertisers"].astype('int')
df["num_users"] = df["num_users"].astype('int')

In [ ]:
ordered_instances = []

for name in instances:
    (V,B) = instances[name]
    num_advertisers, num_users = V.shape
    
    ordered_instances.append((num_advertisers*num_users, name, V, B))

ordered_instances.sort(key=lambda tup:tup[0])
    
for (size, name, V, B) in ordered_instances:
    num_advertisers, num_users = V.shape
    print(size)
    
    alpha1, p1, x1, rt1 = fppe(V,B)
    welfare1 = sum(sum(V * x1))
    revenue1 = sum(p1)
    print(f'FPPE welfare: {welfare1}, revenue: {revenue1}')

    alpha2, p2, x2, rt2 = sppe(V,B)
    welfare2 = sum(sum(V * x2))
    revenue2 = sum(p2)
    print(f'SPPE welfare: {welfare2}, revenue: {revenue2}')

    df.loc[df.name.count()] = {"name": name,
        "num_advertisers": num_advertisers,
               "num_users": num_users,
               "fppe_welfare": welfare1,
               "fppe_rev": revenue1,
               "sppe_welfare": welfare2,
               "sppe_rev": revenue2}

    df.to_pickle(f'./df2/fppe_sppe_{df.name.count()}_df.zip')

## Visualization

In [ ]:
df_vis = pd.read_pickle("./df/fppe_sppe_308_df.zip")
df_vis["welfare_ratio"] = df_vis["fppe_welfare"] / df_vis["sppe_welfare"]
df_vis["rev_ratio"] = df_vis["fppe_rev"] / df_vis["sppe_rev"]

In [ ]:
df_small = df_vis[df_vis["num_advertisers"] >= 3]

x = sorted(df_small["rev_ratio"])
y = [(1.0+i)/len(x) for i in range(len(x))]


plt.rcParams['figure.figsize'] = 1.3*4,4
g3a = sns.lineplot(x, y)
#g3a.set_xlim(0,10)
g3a.set_xlabel("Revenue of FPPE / Revenue of SPPE")
g3a.set_ylabel("Cumulative Frequency")
plt.tight_layout()
plt.savefig("./figs/3a.pdf")

In [ ]:
x = sorted(df_vis["welfare_ratio"])
y = [(1.0+i)/len(x) for i in range(len(x))]

g3b = sns.lineplot(x, y)
g3b.set_xlabel("Welfare of FPPE / Welfare of SPPE")
g3b.set_ylabel("Cumulative Frequency")
plt.tight_layout()
plt.savefig("./figs/3b.pdf")